In [161]:
import requests
import pandas as pd
import numpy as np

In [162]:
#Foresquare credentials
CLIENT_ID = 'XDP05EROIGY2VHJ3RZNRVGCEECJKCBAV0L4JMC4F4SCL0NAF'
CLIENT_SECRET = 'XHKK22SSQYX0REMVWFGNG3GTY5I2HGPPOUERFOEFPIR0YPNO'
VERSION = '20180605'

In [163]:
borough = pd.read_csv('toronto_boroughs.csv')
borough = borough.drop(borough.columns[0], axis=1)
borough['Neighborhood'] = borough['Neighborhood'].str.replace(',', ', ')
borough.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [164]:
locations = pd.read_csv('Geospatial_Coordinates.csv')
locations.columns = ['PostalCode', 'Latitude', 'Longitude']
locations.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [165]:
data = borough.merge(locations, on="PostalCode", how='left')
data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [354]:
data.shape

(103, 5)

# Part 3: exploring Toronto


First of all, let's import the needed packages...

In [166]:
import folium
import matplotlib.cm     as cm
import matplotlib.colors as colors

from geopy.geocoders     import Nominatim
from pandas.io.json      import json_normalize

...and let's have a look at the map of Toronto:

In [167]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [168]:
toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(data['Latitude'], data['Longitude'], data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto)  
    
toronto

We can now go ahead and explore the boroughs of Toronto!

In view of [this](https://www.huffingtonpost.ca/lauren-haw/affordable-toronto-neighbourhoods_b_14682614.html) article published by the _HuffingtonPost_ in February, 2017, we have decided to look closely at **Leslieville**, which appears to be one of the neighborhoods with the highest average townhouse prices among the top 20 most affordable neighborhoods in Toronto.  
In particular, it is neighborhood that has experienced the larger growth during the period 2011-2017.

So, suppose that we want to buy an apartment in **Leslieville**, but the townhouse prices is still too much for us.  

### Let's see whether we can find a cheapest neighborhood that matches Leslieville characteristics!

First of all, we need all the information available about such a neighborhood.  
**Leslieville** is not included in the list of postal codes previously retrieved, but the above article from the _HuffingtonPost_ claims that its postal code is **M4M**, namely the _Studio District_ neighborhood according to Wikipedia. We will stick to such a name for the porpouses of this notebook!

In [318]:
ix = data.index[data['Neighborhood'].str.contains('Studio District')].tolist()[0]
print(f'The neighborhood "Leslieville" is stored at index {ix} of our dataframe!')

The neighborhood "Leslieville" is stored at index 43 of our dataframe!


In [421]:
m4m_index = 43
m4m = data.loc[m4m_index]
m4m

PostalCode                  M4M
Borough            East Toronto
Neighborhood    Studio District
Latitude                43.6595
Longitude              -79.3409
Name: 43, dtype: object

So, **Studio District** is a neighborhood of **East Toronto**!

### Let's explore East Toronto a little bit!

In [171]:
pd.set_option('display.max_colwidth',100)

In [322]:
east_toronto = data[data['Borough']=='East Toronto']
east_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
87,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558


First of all, let's have a look at the map...

In [325]:
address = 'East Toronto, TO'

geolocator = Nominatim()

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.653963, -79.387207.


In [326]:
east_toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(east_toronto['Latitude'], east_toronto['Longitude'], east_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(east_toronto_map)  
    
east_toronto_map

We can begin to explore Toronto to find out if we can find an even more affordable place than those ones in **Studio District**!

In [175]:
def getNearbyVenuesOf(df):
    
    names = df['Neighborhood']
    latitudes = df['Latitude']
    longitudes = df['Longitude']
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We will restrict ourselves to the top 100 venues that are within a radius of 800 meters from the center of each neighborhood.

In [184]:
LIMIT = 100
RADIUS = 800

In [327]:
toronto_venues = getNearbyVenuesOf(data)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [328]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
3,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant
5,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
6,"Rouge, Malvern",43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop
7,"Rouge, Malvern",43.806686,-79.194353,Bus Stop: 85 & 116,43.802198,-79.199389,Bus Station
8,"Rouge, Malvern",43.806686,-79.194353,Ecopainting inc.,43.808417,-79.202392,Construction & Landscaping
9,"Rouge, Malvern",43.806686,-79.194353,Tim Hortons / Esso,43.801660,-79.199133,Coffee Shop


In [329]:
toronto_venues.shape

(3952, 7)

In [333]:
venues_count = toronto_venues.groupby('Neighborhood').count()
venues_count.drop([col for col in venues_count.columns if col!='Venue'], axis=1, inplace=True)
venues_count.columns = ['Total number of venues']
venues_count.sort_values('Total number of venues', ascending=False)

,Total number of venues
Neighborhood,
"Adelaide, King, Richmond",100
Central Bay Street,100
"Harbourfront East, Toronto Islands, Union Station",100
"Chinatown, Grange Park, Kensington Market",100
"Little Portugal, Trinity",100
"Commerce Court, Victoria Hotel",100
Queen's Park,100
Church and Wellesley,100
"Ryerson, Garden District",100


So, **Studio District** counts the third biggest number of venues!

Let's see whether we can find a more affordable neighborhood with as many as that number of venues!

In [334]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot = toronto_onehot[ [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1]) ]

toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [353]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(102, 333)


,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.010000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.010000
1,Agincourt,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles...",0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.045455,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.025000,0.000000,0.00,0.0,0.000000,0.000000,0.025000,0.000000
8,Berczy Park,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [336]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.07
1          Coffee Shop  0.06
2           Steakhouse  0.04
3                Hotel  0.04
4  American Restaurant  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.10
1         Coffee Shop  0.05
2    Malay Restaurant  0.05
3      Sandwich Place  0.05
4     Badminton Court  0.05


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0             BBQ Joint   0.1
1  Fast Food Restaurant   0.1
2                  Park   0.1
3           Pizza Place   0.1
4    Chinese Restaurant   0.1


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0    Pizza Place  0.21
1  Grocery Store  0.21
2    Coffee Shop  0.07
3       Pharmacy  0.07
4     Beer Store  0.07


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.14
1   Coffee Shop  0.07
2

4         Gastropub  0.04


----Emery, Humberlea----
                    venue  freq
0       Convenience Store  0.14
1  Furniture / Home Store  0.14
2          Discount Store  0.14
3            Home Service  0.14
4          Shipping Store  0.14


----Fairview, Henry Farm, Oriole----
                  venue  freq
0           Coffee Shop  0.11
1        Clothing Store  0.11
2  Fast Food Restaurant  0.06
3   Japanese Restaurant  0.06
4         Shopping Mall  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.11
1        Hotel  0.10
2         Café  0.08
3   Restaurant  0.05
4   Steakhouse  0.04


----Flemingdon Park, Don Mills South----
                 venue  freq
0          Coffee Shop  0.08
1           Beer Store  0.08
2  Japanese Restaurant  0.08
3                  Gym  0.08
4     Asian Restaurant  0.08


----Forest Hill North, Forest Hill West----
                venue  freq
0         Coffee Shop  0.19
1  Italian Restaurant  0.12
2  Chinese Res

           venue  freq
0  Deli / Bodega  0.25
1      Cafeteria  0.25
2           Park  0.25
3           Pool  0.25
4  Moving Target  0.00


----St. James Town----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2   Restaurant  0.05
3        Hotel  0.05
4       Bakery  0.04


----Stn A PO Boxes 25 The Esplanade----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.07
2           Restaurant  0.05
3                Hotel  0.04
4  Japanese Restaurant  0.04


----Studio District----
         venue  freq
0       Bakery  0.06
1         Café  0.05
2  Coffee Shop  0.05
3          Bar  0.04
4        Diner  0.04


----The Annex, North Midtown, Yorkville----
                venue  freq
0         Coffee Shop  0.06
1                Café  0.06
2         Pizza Place  0.05
3  Italian Restaurant  0.05
4      Sandwich Place  0.04


----The Beaches----
                 venue  freq
0                  Pub  0.08
1          Coffee Shop  0.06
2                  B

Let's have a closer look at **Studio District**

In [338]:
m4m_venues = toronto_grouped[toronto_grouped['Neighborhood'] == m4m.Neighborhood].T.reset_index()
m4m_venues.columns = ['venue','freq']
m4m_venues = m4m_venues.iloc[1:]
m4m_venues['freq'] = m4m_venues['freq'].astype(float)
m4m_venues = m4m_venues.round({'freq': 2})
m4m_venues = m4m_venues.sort_values('freq', ascending=False).reset_index(drop=True)
m4m_venues = m4m_venues[m4m_venues.freq>0.0]

print('The list of the most frequent venues category in Mimico South:')
print(m4m_venues)
print('\n')

The list of the most frequent venues category in Mimico South:
                        venue  freq
0                      Bakery  0.06
1                        Café  0.05
2                 Coffee Shop  0.05
3                         Bar  0.04
4                       Diner  0.04
5              Sandwich Place  0.04
6                     Brewery  0.03
7                        Park  0.03
8            Sushi Restaurant  0.03
9                   Gastropub  0.03
10        American Restaurant  0.03
11         Italian Restaurant  0.03
12                Pizza Place  0.03
13                 Restaurant  0.02
14  Latin American Restaurant  0.02
15      Vietnamese Restaurant  0.02
16                  Juice Bar  0.02
17                Curling Ice  0.01
18            Coworking Space  0.01
19          Convenience Store  0.01
20                Yoga Studio  0.01
21                  Rock Club  0.01
22             Clothing Store  0.01
23         Chinese Restaurant  0.01
24                Cheese Shop  0.01
2

It seems like there are a lot of café and restaurants...

In [340]:
def get_category(s):
    food = ['Restaurant','Chicken','Pizza','Diner','Bar','Sandwich','Ice Cream', 'Café', 'Dessert', 'Coffee', 'Bakery', 'Gastropub']
    for index,value in s.iteritems():
        if any(w in value for w in food):
            yield 'Food'
        else:
            yield 'Other'

m4m_venues['category'] = list(get_category(m4m_venues['venue']))
m4m_venues.drop('venue', axis=1).groupby('category').count()

,freq
category,
Food,25
Other,31


Wow! Almost one half of venues discovered by _Foresquare_ belongs to the wide category of _Food_.


Ok, now we have all the information that we need.  
It's time to find a less expensive neighborhood in Toronto that matches **Studio District** characteristics!

Namely...
# ...let's make some clustering!

In [437]:
import matplotlib.cm     as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans, DBSCAN

In [483]:
kclusters = 11
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped.drop('Neighborhood', axis=1))

In [484]:
toronto_data = pd.merge(toronto_grouped,data,how='left', on='Neighborhood')
toronto_data.shape

(102, 337)

In [485]:
toronto_data['Cluster Labels'] = kmeans.labels_
print(toronto_data.shape)
toronto_data.head()

(102, 338)


,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,PostalCode,Borough,Latitude,Longitude,Cluster Labels
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.01,M5H,Downtown Toronto,43.650571,-79.384568,2
1,Agincourt,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,M1S,Scarborough,43.794200,-79.262029,2
2,"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,M1V,Scarborough,43.815252,-79.284577,1
3,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,M9V,Etobicoke,43.739416,-79.588437,1
4,"Alderwood, Long Branch",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,M8W,Etobicoke,43.602414,-79.543484,1


In [486]:
toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=11) #, tiles='CartoDB positron'

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
cmap_fill = [colors.rgb2hex(i) for i in cm.Set2_r(np.linspace(0, 1, len(ys)))]
cmap_bord = [colors.rgb2hex(i) for i in cm.Dark2_r(np.linspace(0, 1, len(ys)))]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + '<br><b>Cluster n. ' + str(cluster) + '</b>')
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=cmap_bord[cluster-1],
        fill=True,
        fill_color=cmap_fill[cluster-1],
        fill_opacity=0.7).add_to(toronto_clusters)
       
toronto_clusters

In [487]:
tmp = toronto_data['Cluster Labels'].value_counts()
tmp

2     53
1     20
4      9
0      9
5      5
10     1
9      1
8      1
7      1
6      1
3      1
Name: Cluster Labels, dtype: int64

So, there are other many other neighborhoods that match **Leslieville** characteristics:

In [488]:
m4m_cluster = toronto_data[(toronto_data['Cluster Labels']==1)&(toronto_data['Neighborhood'].index!=m4m_index)][['PostalCode', 'Neighborhood']]\
                    .reset_index(drop=True)\
                    .set_index('PostalCode')
                   
m4m_cluster

,Neighborhood
PostalCode,
M1V,"Agincourt North, L'Amoreaux East, Milliken, Steeles East"
M9V,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles..."
M8W,"Alderwood, Long Branch"
M3H,"Bathurst Manor, Downsview North, Wilson Heights"
M2K,Bayview Village
M1M,"Cliffcrest, Cliffside, Scarborough Village West"
M9B,"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park"
M6M,"Del Ray, Keelesdale, Mount Dennis, Silverthorn"
M3N,Downsview Northwest


Ok, let's see if any of the above neighborhoods is in top 20 most affordable neighborhoods of Toronto:

In [521]:

most_affordable = {'M9V':'300.000-400.000', 'M3N':'300.000-400.000', 'M1E':'300.000-400.000', 'M1B':'300.000-400.000',
                   'M1L':'400.000-495.000', 'M1T':'400.000-495.000', 'M1V':'400.000-495.000', 'M2R':'400.000-495.000',
                   'M6K':'495.000-540.000', 'M1P':'495.000-540.000', 'M1W':'495.000-540.000', 'M1S':'495.000-540.000',
                   'M2H':'540.000-600.000', 'M2J':'540.000-600.000', 'M6P':'540.000-600.000', 'M4Y':'540.000-600.000',
                   'M4E':'600.000-640.000', 'M6H':'600.000-640.000', 'M8V':'600.000-640.000'}

In [522]:
townhouse_prices = pd.Series(most_affordable).to_frame()
townhouse_prices.columns = ['Average Townhouse Prices (in CAD)']
townhouse_prices

,Average Townhouse Prices (in CAD)
M9V,300.000-400.000
M3N,300.000-400.000
M1E,300.000-400.000
M1B,300.000-400.000
M1L,400.000-495.000
M1T,400.000-495.000
M1V,400.000-495.000
M2R,400.000-495.000
M6K,495.000-540.000
M1P,495.000-540.000


In [523]:
results = pd.merge(townhouse_prices,m4m_cluster,how='inner',left_index=True,right_index=True)
results

,Average Townhouse Prices (in CAD),Neighborhood
M9V,300.000-400.000,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles..."
M3N,300.000-400.000,Downsview Northwest
M1V,400.000-495.000,"Agincourt North, L'Amoreaux East, Milliken, Steeles East"
M1W,495.000-540.000,"L'Amoreaux West, Steeles West"
M2H,540.000-600.000,Hillcrest Village


As a result, we may to look at the following neighborhoods if we want a less expensive house yet matching **Leslieville** criteria:

In [536]:
pd.set_option('display.max_colwidth',150)

In [537]:
results[results['Average Townhouse Prices (in CAD)'].str.contains('300.000')]['Neighborhood'].to_frame()

,Neighborhood
M9V,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown"
M3N,Downsview Northwest
